In [107]:
import numpy as np
import pandas as pd

In [108]:
df = pd.read_csv('/content/diabetes.csv')

In [109]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [110]:
df.corr()['Outcome'] # hamm dekhenge kon sa cheez outcome ko jyda effect kar raha hai.

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [111]:
X = df.iloc[:,:-1].values #we are extracting X and y values.
y = df.iloc[:,-1].values

In [112]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [113]:
X = scaler.fit_transform(X) # scaling the data.

In [114]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [115]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [157]:
#let us now form a model.
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [117]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim = 8))#yaha aapko khud s decide krna hai kitne layers honge aur kitne nodes honge.
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [118]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.4375 - loss: 0.7301 - val_accuracy: 0.5260 - val_loss: 0.6754
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5942 - loss: 0.6599 - val_accuracy: 0.6753 - val_loss: 0.6182
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6967 - loss: 0.6182 - val_accuracy: 0.7338 - val_loss: 0.5794
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7397 - loss: 0.5784 - val_accuracy: 0.7922 - val_loss: 0.5514
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7771 - loss: 0.5380 - val_accuracy: 0.7987 - val_loss: 0.5309
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7527 - loss: 0.5345 - val_accuracy: 0.8052 - val_loss: 0.5156
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7597 - loss: 0.5176 - val_accuracy: 0.8052 - val_loss: 0.5059
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7777 - loss: 0.4969 - val_accuracy: 0.7987

# we will learn how to select appropriate optimizer.
# Number of nodes in a layer.
#How to seelect number of hidden layer

In [119]:
# Install the Keras Tuner library
!pip install keras-tuner

In [120]:
import kerastuner as kt

In [121]:
#First of all we will make a function.
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [122]:
#Now we will make our tuner object.
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [123]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [124]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [125]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [126]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [127]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7697 - loss: 0.5389 - val_accuracy: 0.7792 - val_loss: 0.5072
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7858 - loss: 0.5100 - val_accuracy: 0.7857 - val_loss: 0.4946
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7576 - loss: 0.5168 - val_accuracy: 0.7727 - val_loss: 0.4826
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7738 - loss: 0.4898 - val_accuracy: 0.7727 - val_loss: 0.4758
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7812 - loss: 0.4740 - val_accuracy: 0.7727 - val_loss: 0.4717
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7646 - loss: 0.4707 - val_accuracy: 0.7727 - val_loss: 0.4675
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7918 - loss: 0.4452 - val_accuracy: 0.7662 - val_loss: 0.4663
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8003 - loss: 0.4445 - val_accuracy: 0.7662 - 

#Next we will check what is the write number of neuron in the given layer.

In [128]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units',min_value = 8,max_value = 128)#Ye multiple model build krega initial 8 lega aur 128 tak jayega ur best hame dega.
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [129]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='UP')

Reloading Tuner from mydir/UP/tuner0.json


In [130]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [131]:
tuner.get_best_hyperparameters()[0].values

{'units': 32}

In [133]:
model = tuner.get_best_models(num_models=1)[0]

In [134]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7635 - loss: 0.5357
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7776 - loss: 0.4987 
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7389 - loss: 0.5108 
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7651 - loss: 0.4950 
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7840 - loss: 0.4645 
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7763 - loss: 0.4816 
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7532 - loss: 0.4887 
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7787 - loss: 0.4684 
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7577 - loss: 0.4638 
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7678 - loss: 0.4654 
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8018 - loss: 0.4537 
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/

# How to select number of layers.

In [140]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  for i in range(hp.Int('num_layer',min_value=1,max_value=10)):
    model.add(Dense(32,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [141]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='UP_layers')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [142]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 13s


In [143]:
best_hyperparameters = tuner.get_best_hyperparameters()[0].values
best_layers = best_hyperparameters['num_layer']
print(f"The best number of layers is: {best_layers}")

The best number of layers is: 7


In [144]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [146]:
model.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.7534 - loss: 0.4892 - val_accuracy: 0.7338 - val_loss: 0.4744
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7800 - loss: 0.4463 - val_accuracy: 0.7532 - val_loss: 0.4739
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7869 - loss: 0.4483 - val_accuracy: 0.7727 - val_loss: 0.4639
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7956 - loss: 0.4478 - val_accuracy: 0.7987 - val_loss: 0.4543
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7750 - loss: 0.4462 - val_accuracy: 0.7468 - val_loss: 0.4910
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7666 - loss: 0.4569 - val_accuracy: 0.7922 - val_loss: 0.4612
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7950 - loss: 0.4481 - val_accuracy: 0.8052 - val_loss: 0.4623
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7924 - loss: 0.4403 - val_accuracy

In [159]:
def build_model(hp):
  model = Sequential()
  counter = 0
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter == 0 :
      model.add(
          Dense(
              hp.Int('units' + str(i),min_value = 8,max_value=12,step = 8),
              activation=hp.Choice('activation'+ str(i), values=['relu','tanh','sigmoid']),
              input_dim = 8
              )
          )
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(
          Dense(
              hp.Int('units' + str(i),min_value = 8,max_value=12,sttep = 8),
              activation=hp.Choice('activation'+ str(i), values=['relu','tanh','sigmoid']),

              )
          )
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
      counter += 1 #Andar wale sare layers ho gaye.


  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model



In [161]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='fina1')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [162]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.5844155550003052

Best val_accuracy So Far: 0.6688311696052551
Total elapsed time: 00h 00m 16s


In [163]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 8,
 'activation0': 'tanh',
 'dropout0': 0.7,
 'optimizer': 'rmsprop',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1}

In [164]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [165]:
model.fit(X_train,y_train,epochs=200,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6337 - loss: 0.6853 - val_accuracy: 0.6753 - val_loss: 0.6096
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6023 - loss: 0.6966 - val_accuracy: 0.6948 - val_loss: 0.5996
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6654 - loss: 0.6269 - val_accuracy: 0.7013 - val_loss: 0.5922
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6372 - loss: 0.6497 - val_accuracy: 0.7273 - val_loss: 0.5855
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6935 - loss: 0.6229 - val_accuracy: 0.7532 - val_loss: 0.5754
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6697 - loss: 0.6049 - val_accuracy: 0.7597 - val_loss: 0.5676
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6702 - loss: 0.6077 - val_accuracy: 0.7468 - val_loss: 0.5602
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6469 - loss: 0.6036 - val_accuracy: 0.75